# Document Summarization

## Introduction 
This demo showcases a chatbot system powered by Generative AI (OpenAI). Using technologies like <b>RAG, Langchain, and LLM models</b> users can ask questions in simple terms, retrieve relevant data, and receive concise answers. The approach integrates retrieval-based and generative techniques to deliver accurate, user-friendly insights from structured sources.

The following diagram illustrates the overall architecture.

<center><img src="images/header_chat_td.png" alt="architecture" /></center>

# Steps in the analysis
1. Configuring the environment  
2. Connect to Vantage  
3. Data Exploration  
4. Generate the embeddings  
5. Load the existing embeddings to DB  
6. Calculate the VectorDistance using Teradata Vantage in-DB function  
7. LLM  
8. Chat with documents  
9. Cleanup  
